<a href="https://colab.research.google.com/github/smBello-tse/CIFAR10-classification/blob/main/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>